In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Eval")

LangSmith 추적을 시작합니다.
[프로젝트명]
Eval


In [6]:
import pandas as pd

In [8]:
df = pd.read_csv("outputs/synthetic_out.csv")
df

# Convert DataFrame to examples scheme
examples = [
    {
        "inputs": {"question": row["query"]},
        "outputs": {"answer": row["answers"]},
    }
    for _, row in df.iterrows()
]

print(f"Created {len(examples)} examples")
examples[:2]  # Show first 2 examples

Created 107 examples


[{'inputs': {'question': 'Who is responsible for publishing this work?'},
  'outputs': {'answer': 'The Secretary-General of the OECD.'}},
 {'inputs': {'question': 'What are three atomic facts about Cyprus and Turkiye mentioned in the document?'},
  'outputs': {'answer': 'There is no single authority representing both Turkish and Greek Cypriot people on the Island; Turkiye recognises the Turkish Republic of Northern Cyprus (TRNC); The Republic of Cyprus is recognised by all United Nations members with the exception of T�rkiye.'}}]

In [9]:
from langsmith import Client

client = Client()

# Create the dataset and examples in LangSmith
dataset_name = "RAG eval dataset"
dataset = client.create_dataset(dataset_name=dataset_name)
client.create_examples(
    dataset_id=dataset.id,
    examples=examples
)

{'example_ids': ['cdf372d6-397f-4bb4-ab9a-2cf52891a07d',
  '67f60b40-7e0a-49f9-817e-cecd5eea7f5c',
  '6ba5b5a7-a0dc-453f-ae3e-452bc765d20f',
  'ef816b23-87f7-4f5f-95bc-6b664175a8d7',
  'cd5bc95e-c111-43ef-a8a6-0be2c4902fa8',
  'd4fd1c86-b17f-43a5-9f52-bd3351d8c772',
  'f867c518-c265-4888-b759-8ad52c3b7199',
  '87c73137-0c9e-4cfc-83ab-fdc495d40451',
  '2d685e23-e793-4793-9300-1d48da3ec4f2',
  '4915398d-f0b7-4fcc-9c72-525b787c2726',
  '5b15abbd-3d10-429b-ae3d-bdf6874eabf4',
  'e5cf84f7-0300-4a2e-abc1-97bd78b98be1',
  'f3f1966f-bcee-4657-8680-6e4b622e69d2',
  '1edce126-609f-4dab-ac00-2e31e9128255',
  '92607d02-432a-457b-b058-721cee593e0a',
  '5b5e8f66-234a-441d-a47d-fafc3289fc8e',
  '439992a6-98ec-4256-aaf6-96d35559e3e4',
  'd38bc2d5-caf1-40ce-be77-ef799be96115',
  '10bc8ce6-0294-4e81-9045-312fc1bcf398',
  '4dd91ef9-4e90-43c5-8a66-b96cb46cec5d',
  '2406805c-49b5-40a7-b0a2-aa62ce23d0ae',
  '9406d909-817b-478b-88e3-400e8faec5bb',
  'be25a33c-787f-49c4-8cae-4c892bc68e80',
  'e4bedbcf-d22a-43

In [12]:
import pickle
from langchain.schema import Document
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [13]:
from pydantic import BaseModel, Field
class RetrievalResult(BaseModel):
    """Retrieval result with details."""
    query: str = Field(..., description="The query of the data")
    answers: str = Field(..., description="The answers of the data")
    chunk_id: str = Field(..., description="The chunk id of the evidence")
    page_number: str = Field(..., description="The page number of the evidence")
    evidence: str = Field(..., description="The evidence of the data")

In [14]:
def format_context(results: list[Document]) -> str:
    arr = []
    for i, doc in enumerate(results):
        text = f"""Retrieved #{i+1}
                    {doc.page_content}
                    {doc.metadata['text_summary'] if len(doc.metadata['text_summary']) > 0 else ''}
                    {doc.metadata['image_summary'] if len(doc.metadata['image_summary']) > 0 else ''}
                    chunk_id: {doc.metadata['id']}
                    page number: {doc.metadata['page']}
                    
                """
        arr.append(text)
    return "\n".join(arr)

In [15]:
with open("outputs/split_documents.pkl", "rb") as f:
    split_documents = pickle.load(f)
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

vectorstore = FAISS.load_local(
    "faiss_index", 
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True  # needed in newer versions
)
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
bm25_retriever = BM25Retriever.from_documents(
    split_documents,
)
bm25_retriever.k = 1
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
)

# GraphState 상태 정의
class GraphState(TypedDict):
    question: Annotated[str, "Question"]  # 질문
    context: Annotated[str, "Context"]  # 문서의 검색 결과
    answer: Annotated[RetrievalResult, "Answer"]  # 답변
    messages: Annotated[list, add_messages]  # 메시지(누적되는 list)
# 문서 검색 노드
def retrieve_document(state: GraphState) -> GraphState:
    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 문서에서 검색하여 관련성 있는 문서를 찾습니다.
    retrieved_docs = ensemble_retriever.invoke(latest_question)

    # 검색된 문서를 형식화합니다.(프롬프트 입력으로 넣어주기 위함)
    context = format_context(retrieved_docs)

    # 검색된 문서를 context 키에 저장합니다.
    return {"context": context}


# 답변 생성 노드
def llm_answer(state: GraphState) -> GraphState:
    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 검색된 문서를 상태에서 가져옵니다.
    context = state["context"]
   
    llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)
      
    system_prompt = """You are an assistant for question-answering tasks. 
        Use the following pieces of retrieved context to answer the question. 
        If you don't know the answer, just say that you don't know. 
        Answer in Korean.
    """
    
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Context: " + context},
        {"role": "user", "content": "Question: " + latest_question},
    ]
        
    # 체인을 호출하여 답변을 생성합니다.
    model_with_structure = llm.with_structured_output(RetrievalResult)
    response = model_with_structure.invoke(prompt)
    print("response", response)
    # 생성된 답변, (유저의 질문, 답변) 메시지를 상태에 저장합니다.
    return {
        "answer": response,
        "messages": [("user", latest_question), ("assistant", response.answers)],
    }
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

# 그래프 생성
workflow = StateGraph(GraphState)

# 노드 정의
workflow.add_node("retrieve", retrieve_document)
workflow.add_node("llm_answer", llm_answer)

# 엣지 정의
workflow.add_edge("retrieve", "llm_answer")  # 검색 -> 답변
workflow.add_edge("llm_answer", END)  # 답변 -> 종료

# 그래프 진입점 설정
workflow.set_entry_point("retrieve")

# 체크포인터 설정
memory = MemorySaver()

# 컴파일
app = workflow.compile(checkpointer=memory)

In [16]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import invoke_graph, stream_graph, random_uuid

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

# 질문 입력
inputs = GraphState(question="터키가 직면한 위기와 도전을 알려주세요.")

In [17]:
inputs

{'question': '터키가 직면한 위기와 도전을 알려주세요.'}

In [28]:
result = app.invoke(inputs, config)

response query='터키가 직면한 위기와 도전 알려주세요' answers='제공된 자료( OECD Interim Report 및 ‘Note by the Republic of Turkiye’ )에 따르면 터키가 직면한 주요 이슈와 도전은 다음과 같습니다.\n\n1) 키프로스 관련 외교·정치 문제\n- 터키는 키프로스 섬 문제와 관련해 북키프로스 터키 공화국(TRNC)을 인정하고 있으며, 문서에서 터키는 현 입장을 유지하겠다고 밝히고 있습니다.(키프로스에 대한 주권·인정 문제로 인한 외교적 긴장과 분쟁이 지속되는 점)\n\n2) 거시경제·정책적 도전(보고서에서 모든 국가에 권고되는 공통 과제로서 터키에도 적용됨)\n- 글로벌 무역체계 내 협력 필요: 무역정책의 투명성과 예측가능성 제고를 위해 다른 국가들과 협력해야 함. \n- 통화정책 운용의 어려움: 중앙은행은 인플레이션이 목표로 완화될 것으로 예상될 때 정책금리를 낮출 수 있으나, 인플레이션 기대치가 잘 고정되어 있어야 하므로 물가안정 관리가 중요함.\n- 재정 규율 유지: 장기적 재정건전성(부채 지속가능성)을 지키기 위한 재정 규율이 필요하며, 그래야 향후 충격에 대응할 재원이 확보됨.\n- 구조개혁 필요성: 생활수준을 지속적으로 개선하고 인공지능 등 신기술의 잠재력을 실현하기 위해 강화된 구조개혁이 요구됨.\n\n참고: 위 내용은 제공된 OECD 문서의 요지에 근거한 것이며, 일부는 모든 국가에 대한 일반 권고로서 터키에 적용되는 도전들입니다.' chunk_id='108, 3' page_number='24, 4' evidence='(Chunk 108, p.24) "countries need to find ways of engaging co‑operatively within the global trading system and working together to make trade policy more transparent and predictable. Central banks should remai

In [34]:
result['answer'].answers, result['answer'].evidence

('제공된 자료( OECD Interim Report 및 ‘Note by the Republic of Turkiye’ )에 따르면 터키가 직면한 주요 이슈와 도전은 다음과 같습니다.\n\n1) 키프로스 관련 외교·정치 문제\n- 터키는 키프로스 섬 문제와 관련해 북키프로스 터키 공화국(TRNC)을 인정하고 있으며, 문서에서 터키는 현 입장을 유지하겠다고 밝히고 있습니다.(키프로스에 대한 주권·인정 문제로 인한 외교적 긴장과 분쟁이 지속되는 점)\n\n2) 거시경제·정책적 도전(보고서에서 모든 국가에 권고되는 공통 과제로서 터키에도 적용됨)\n- 글로벌 무역체계 내 협력 필요: 무역정책의 투명성과 예측가능성 제고를 위해 다른 국가들과 협력해야 함. \n- 통화정책 운용의 어려움: 중앙은행은 인플레이션이 목표로 완화될 것으로 예상될 때 정책금리를 낮출 수 있으나, 인플레이션 기대치가 잘 고정되어 있어야 하므로 물가안정 관리가 중요함.\n- 재정 규율 유지: 장기적 재정건전성(부채 지속가능성)을 지키기 위한 재정 규율이 필요하며, 그래야 향후 충격에 대응할 재원이 확보됨.\n- 구조개혁 필요성: 생활수준을 지속적으로 개선하고 인공지능 등 신기술의 잠재력을 실현하기 위해 강화된 구조개혁이 요구됨.\n\n참고: 위 내용은 제공된 OECD 문서의 요지에 근거한 것이며, 일부는 모든 국가에 대한 일반 권고로서 터키에 적용되는 도전들입니다.',
 '(Chunk 108, p.24) "countries need to find ways of engaging co‑operatively within the global trading system and working together to make trade policy more transparent and predictable. Central banks should remain vigilant, but can lower policy interest rates where underlying inflation is projected to

In [ ]:
def rag_bot(question: str) -> dict:
    inputs = GraphState(question=question)
    res = invoke_graph(app, inputs, config)
    return {"answer": res['answer'].answers, "document": res['answer'].evidence}

In [19]:
from typing_extensions import Annotated, TypedDict

# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

# Grade prompt
correctness_instructions = """You are a teacher grading a quiz. You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. (2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the student's answer meets all of the criteria.
A correctness value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
grader_llm = ChatOpenAI(model="gpt-5-nano", temperature=0).with_structured_output(
    CorrectnessGrade, method="json_schema", strict=True
)

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """An evaluator for RAG answer accuracy"""
    answers = f"""\
QUESTION: {inputs['question']}
GROUND TRUTH ANSWER: {reference_outputs['answer']}
STUDENT ANSWER: {outputs['answer']}"""
    # Run evaluator
    grade = grader_llm.invoke([
        {"role": "system", "content": correctness_instructions},
        {"role": "user", "content": answers}
    ])
    return grade["correct"]

In [20]:
# Grade output schema
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[
        bool, ..., "Provide the score on whether the answer addresses the question"
    ]

# Grade prompt
relevance_instructions = """You are a teacher grading a quiz. You will be given a QUESTION and a STUDENT ANSWER. Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
relevance_llm = ChatOpenAI(model="gpt-5-nano", temperature=0).with_structured_output(
    RelevanceGrade, method="json_schema", strict=True
)

# Evaluator
def relevance(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer helpfulness."""
    answer = f"QUESTION: {inputs['question']}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = relevance_llm.invoke([
        {"role": "system", "content": relevance_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

In [21]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[
        bool, ..., "Provide the score on if the answer hallucinates from the documents"
    ]

# Grade prompt
grounded_instructions = """You are a teacher grading a quiz. You will be given FACTS and a STUDENT ANSWER. Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is grounded in the FACTS. (2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
grounded_llm = ChatOpenAI(model="gpt-5-nano", temperature=0).with_structured_output(
    GroundedGrade, method="json_schema", strict=True
)

# Evaluator
def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = grounded_llm.invoke([
        {"role": "system", "content": grounded_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["grounded"]

In [22]:
# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[
        bool,
        ...,
        "True if the retrieved documents are relevant to the question, False otherwise",
    ]

# Grade prompt
retrieval_relevance_instructions = """You are a teacher grading a quiz. You will be given a QUESTION and a set of FACTS provided by the student. Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
retrieval_relevance_llm = ChatOpenAI(
    model="gpt-5-nano", temperature=0
).with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=True)

def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """An evaluator for document relevance"""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nQUESTION: {inputs['question']}"
    # Run evaluator
    grade = retrieval_relevance_llm.invoke([
        {"role": "system", "content": retrieval_relevance_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

In [24]:
res = rag_bot("터키가 직면한 위기와 도전을 알려주세요.")
res


🔄 Node: retrieve 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
context:
Retrieved #1
                    The Interim Report says that countries need to find ways of engaging co‑operatively within the global trading system  
and working together to make trade policy more transparent and predictable. Central banks should remain vigilant, but  
can lower policy interest rates where underlying inflation is projected to moderate towards target, provided inflation  
expectations remain well anchored. Fiscal discipline is needed to safeguard longer‑term debt sustainability and allow  
governments to react to future shocks. Enhanced structural reform efforts are required to durably improve living  
standards and help realise the potential from new technologies, such as artificial intelligence.

  

The Interim Report is an update on the assessment in the June 2025 of the Economic Outlook (Volume 2025 Issue 1).

PDF ISBN 978-92-64-57606-3

  

9HSTCQE\*fhgagd+
                    
     

TypeError: 'NoneType' object is not subscriptable

In [23]:
def target(inputs: dict) -> dict:
    return rag_bot(inputs["question"])

experiment_results = client.evaluate(
    target,
    data=dataset_name,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix="rag-doc-relevance",
    metadata={"version": "langgraph, gpt-5-nano"},
)

# Explore results locally as a dataframe if you have pandas installed
# experiment_results.to_pandas()

View the evaluation results for experiment: 'rag-doc-relevance-a23823da' at:
https://smith.langchain.com/o/824afa80-3494-5b99-80b3-21be59fa9f24/datasets/69895610-d8c4-448d-9c65-1aa0f914c6c4/compare?selectedSessions=0141f0ef-917b-4c25-8ccd-b98eecdd0e46




0it [00:00, ?it/s]


🔄 Node: retrieve 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
context:
Retrieved #1
                    17. Aggregate consumer price inflation for the G20 countries is expected to decline as economic  
growth and labour markets ease (Figure 8, Panel B). In the United States, the increase in effective tariff  
rates will further boost inflation, with the rate of pass-through to final goods prices assumed to strengthen  
as businesses become less willing to absorb the rising cost of imported goods. As a result, US annual  
inflation is projected to remain above target throughout 2026. In the euro area, inflation is anticipated to  
remain subdued, while the normalisation of food prices in Japan will contribute to inflation moving back to  
the central bank target in 2026. Overall, headline inflation in the G20 advanced economies is projected to  
be 2.5% in both 2025 and 2026, while core inflation is also anticipated to remain broadly stable at 2.6% in  
2025 and 2.5% in 2026. I

Error running target function: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "/home/jake/DP/.venv/lib/python3.10/site-packages/langsmith/evaluation/_runner.py", line 1923, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/home/jake/DP/.venv/lib/python3.10/site-packages/langsmith/run_helpers.py", line 693, in wrapper
    function_result = run_container["context"].run(
  File "/tmp/ipykernel_14713/298657276.py", line 2, in target
    return rag_bot(inputs["question"])
  File "/tmp/ipykernel_14713/452748136.py", line 4, in rag_bot
    return {"answer": res['answer'], "document": res['answer'].evidence}
TypeError: 'NoneType' object is not subscriptable
Error running evaluator <DynamicRunEvaluator correctness> on run a3af61ab-e53b-402f-8d64-6dafb2482a68: KeyError('answer')
Traceback (most recent call last):
  File "/home/jake/DP/.venv/lib/python3.10/site-packages/langsmith/evaluation/_runner.py", line 1619, in _run_evaluators
    evaluato

response query="Identify three key facts about growth projections for advanced G20 economies mentioned in the document, including (1) US GDP growth trajectory 2024-2026, (2) reasons for the US slowdown, and (3) Canada's projected growth for 2025 and 2026." answers='1) 미국의 연간 GDP 성장률은 2024년 2.8%에서 2025년 1.8%, 2026년 1.5%로 둔화될 것으로 전망됩니다.\n2) 미국 경기 둔화의 주요 원인으로는 유효 관세율 상승(무역비용 증가), 정책 불확실성 고조, 순이민 감소, 연방 인력 감축 등이 지목됩니다.\n3) 캐나다의 예상 성장률은 2025년에 1.1%, 2026년에 1.2%로 비교적 완만한 성장세를 보일 것으로 전망됩니다.' chunk_id='62' page_number='17' evidence='"Annual GDP growth is projected to slow from 2.8% in 2024 to 1.8% in 2025 and 1.5% in 2026." \n"Growth is expected to slow in the United States as higher effective tariff rates further come into effect and policy uncertainty remains elevated. A drop in net immigration and reductions in the federal workforce are also anticipated to soften economic growth." \n"In Canada, growth is projected to remain modest, at 1.1% in 2025 and 1.2% in 2026, given greater trade restr

...

Error running target function: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "/home/jake/DP/.venv/lib/python3.10/site-packages/langsmith/evaluation/_runner.py", line 1923, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/home/jake/DP/.venv/lib/python3.10/site-packages/langsmith/run_helpers.py", line 693, in wrapper
    function_result = run_container["context"].run(
  File "/tmp/ipykernel_14713/298657276.py", line 2, in target
    return rag_bot(inputs["question"])
  File "/tmp/ipykernel_14713/452748136.py", line 4, in rag_bot
    return {"answer": res['answer'], "document": res['answer'].evidence}
TypeError: 'NoneType' object is not subscriptable
Error running evaluator <DynamicRunEvaluator correctness> on run b98eb250-4633-42d9-b255-1e8449814d68: KeyError('answer')
Traceback (most recent call last):
  File "/home/jake/DP/.venv/lib/python3.10/site-packages/langsmith/evaluation/_runner.py", line 1619, in _run_evaluators
    evaluato

response query='item_01에 대해 차트에 표시된 세 가지 데이터 포인트는 무엇인가요?' answers='item_01의 값: 4.5%, 3.5%, 2.0%' chunk_id='102' page_number='23' evidence='표에서 item_01 행: Baseline = 4.5%, Baseline_with_structural_reforms = 3.5%, With_structural_reforms_with_structural_reforms_and_faster_AI_adoption = 2.0%'

🔄 Node: llm_answer 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
answer:
query='item_01에 대해 차트에 표시된 세 가지 데이터 포인트는 무엇인가요?' answers='item_01의 값: 4.5%, 3.5%, 2.0%' chunk_id='102' page_number='23' evidence='표에서 item_01 행: Baseline = 4.5%, Baseline_with_structural_reforms = 3.5%, With_structural_reforms_with_structural_reforms_and_faster_AI_adoption = 2.0%'
('user', 'What are three data points shown for item_01 in the chart?')
('assistant', 'item_01의 값: 4.5%, 3.5%, 2.0%')

🔄 Node: retrieve 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
context:
Retrieved #1
                    and reallocate spending and enhance revenues are key to ensuring debt burdens stabilise.  
· Enhanced structural ref

Error running target function: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "/home/jake/DP/.venv/lib/python3.10/site-packages/langsmith/evaluation/_runner.py", line 1923, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/home/jake/DP/.venv/lib/python3.10/site-packages/langsmith/run_helpers.py", line 693, in wrapper
    function_result = run_container["context"].run(
  File "/tmp/ipykernel_14713/298657276.py", line 2, in target
    return rag_bot(inputs["question"])
  File "/tmp/ipykernel_14713/452748136.py", line 4, in rag_bot
    return {"answer": res['answer'], "document": res['answer'].evidence}
TypeError: 'NoneType' object is not subscriptable
Error running evaluator <DynamicRunEvaluator correctness> on run 8a69ba6f-de73-4650-8b82-454c94905f7d: KeyError('answer')
Traceback (most recent call last):
  File "/home/jake/DP/.venv/lib/python3.10/site-packages/langsmith/evaluation/_runner.py", line 1619, in _run_evaluators
    evaluato

response query='What is key to ensuring debt burdens stabilise?' answers='지출을 재배분하고 수입(세수)을 확대하는 것이 부채 부담을 안정시키는 데 핵심입니다.' chunk_id='18' page_number='7' evidence='and reallocate spending and enhance revenues are key to ensuring debt burdens stabilise.'

🔄 Node: llm_answer 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
answer:
query='What is key to ensuring debt burdens stabilise?' answers='지출을 재배분하고 수입(세수)을 확대하는 것이 부채 부담을 안정시키는 데 핵심입니다.' chunk_id='18' page_number='7' evidence='and reallocate spending and enhance revenues are key to ensuring debt burdens stabilise.'
('user', 'What is key to ensuring debt burdens stabilise?')
('assistant', '지출을 재배분하고 수입(세수)을 확대하는 것이 부채 부담을 안정시키는 데 핵심입니다.')

🔄 Node: retrieve 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
context:
Retrieved #1
                    should continue efforts to develop and update regulations related to crypto assets. Tailored regulatory  
approaches are warranted while ensuring regulations maintain technological neu

KeyboardInterrupt: 